In [2]:
import numpy as np
import pandas as pd

import tarfile
import gzip
import re
import os
import datetime as dt

In [3]:
year_num = 10 # Number of past years to consider
extremes_num = 10 # Number of hottest and coldest places to display

In [4]:
yearfiles = os.listdir("./gsod_all_years")
yearfiles.sort()
yearfiles = yearfiles[-year_num:]
years = [int(re.findall('\d+',yearfile)[0]) for yearfile in yearfiles]

FileNotFoundError: [WinError 3] The system cannot find the path specified: './gsod_all_years'

In [4]:
station_loc = pd.read_csv('isd-history.csv')
station_loc = station_loc.replace([0.0, -999.0, -999.9],np.nan)
station_loc = station_loc[pd.notnull(station_loc['LAT']) & pd.notnull(station_loc['LON'])]
station_loc = station_loc[[int(re.findall('^\d{4}', str(end_year))[0])==max(years) for end_year in station_loc['END']]]
station_loc = station_loc[[int(re.findall('^\d{4}', str(beg_year))[0])<=min(years) for beg_year in station_loc['BEGIN']]]

In [5]:
station_loc['LBL'] = station_loc[['STATION NAME','STATE','CTRY']].apply(lambda x: x.str.cat(sep=', '), axis=1)
station_loc['ELEV_LBL'] = station_loc['ELEV(M)'].apply(lambda x: 'Elevation: '+str(x)+' m' if ~np.isnan(x) else np.nan)
station_loc['LBL'] = station_loc[['LBL','ELEV_LBL']].apply(lambda x: x.str.cat(sep='<br>'), axis=1)
station_loc = station_loc.drop(['STATION NAME','STATE','ELEV_LBL','ICAO','BEGIN','END'], axis=1)
#station_loc = station_loc.sample(stat_num)

In [6]:
df = pd.DataFrame([])
df_day = pd.DataFrame([])

def preprocess_station_file_content(content):
    headers=content.pop(0)
    headers=[headers[ind] for ind in [0,1,2,3,4,8,11,12]]
    for d in range(len(content)):
        content[d]=[content[d][ind] for ind in [0,1,2,3,5,13,17,18]]
    content=pd.DataFrame(content, columns=headers)
    content.rename(columns={'STN---': 'USAF'}, inplace=True)
    content['MAX'] = content['MAX'].apply(lambda x: re.sub("\*$","",x))
    content['MIN'] = content['MIN'].apply(lambda x: re.sub("\*$","",x))
    content[['WBAN','TEMP','DEWP','WDSP','MAX','MIN']] = content[['WBAN','TEMP','DEWP','WDSP','MAX','MIN']].apply(pd.to_numeric)
    content['YEARMODA']=pd.to_datetime(content['YEARMODA'], format='%Y%m%d', errors='ignore')
    content['YEAR']=pd.DatetimeIndex(content['YEARMODA']).year
    content['MONTH']=pd.DatetimeIndex(content['YEARMODA']).month
    content['DAY']=pd.DatetimeIndex(content['YEARMODA']).day
    return content

In [7]:
# yearfile = yearfiles[-1]
# print(yearfile)
# i=0
# tar = tarfile.open("./gsod_all_years/"+yearfile, "r")
# print(len(tar.getmembers()[1:]))
# #for member in np.random.choice(tar.getmembers()[1:], size=stat_num, replace=False):
# for member in tar.getmembers()[1:]:
#     name_parts = re.sub("\.op\.gz$","",re.sub("^\./","",member.name)).split("-")
#     usaf = name_parts[0]
#     wban = int(name_parts[1])
#     if station_loc[(station_loc['USAF']==usaf) & (station_loc['WBAN']==wban)].shape[0]!=0:
#         i=i+1
#         #if i%(stat_num//10) == 0: print(i)
#         f=tar.extractfile(member)
#         f=gzip.open(f, 'rb')
#         content=[re.sub(" +", ",", line.decode("utf-8")).split(",") for line in f.readlines()]
#         content=preprocess_station_file_content(content)
#         df_day = df_day.append(content[content['YEARMODA']==content['YEARMODA'].max()])
#         content = content.groupby(['USAF','WBAN','YEAR','MONTH']).agg('median').reset_index()
#         df = df.append(content)
# tar.close()

In [8]:
# df

In [9]:
# station_loc

In [10]:
# merged_df = pd.merge(df, station_loc, how="right", on="USAF")
# merged_df

In [11]:
# merged_clean_df = merged_df.loc[merged_df["USAF"] != "999999", :]
# merged_clean_df

In [12]:
# merged_clean_df.to_csv('merged_cleaned_dataframe.csv',index=False)

In [14]:
# day = df_day['YEARMODA'].max()
# df_day = df_day[df_day['YEARMODA']==day]

In [20]:
for yearfile in yearfiles[:-1]:
    print(yearfile)
    i=0
    tar = tarfile.open("./gsod_all_years/"+yearfile, "r")
    print(len(tar.getmembers()[1:]))
    #for member in np.random.choice(tar.getmembers()[1:], size=stat_num, replace=False):
    for member in tar.getmembers()[1:]:
        name_parts = re.sub("\.op\.gz$","",re.sub("^\./","",member.name)).split("-")
        usaf = name_parts[0]
        wban = int(name_parts[1])
        if station_loc[(station_loc['USAF']==usaf) & (station_loc['WBAN']==wban)].shape[0]!=0:
            i=i+1
            #if i%(stat_num//10) == 0: print(i)
            f=tar.extractfile(member)
            f=gzip.open(f, 'rb')
            content=[re.sub(" +", ",", line.decode("utf-8")).split(",") for line in f.readlines()]
            content=preprocess_station_file_content(content)
            df_day = df_day.append(content[content['YEARMODA']==content['YEARMODA'].max()])
            content = content.groupby(['USAF','WBAN','YEAR','MONTH']).agg('median').reset_index()
            years_df = df.append(content)
    tar.close()

gsod_2010.tar
11238
gsod_2011.tar
10702
gsod_2012.tar
11517
gsod_2013.tar
11580
gsod_2014.tar
11540
gsod_2015.tar
11664
gsod_2016.tar
12076
gsod_2017.tar
11923
gsod_2018.tar
12875


In [41]:
df_day

,USAF,WBAN,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY
364,161490,99999,2010-12-31,40.7,38.8,1.8,42.8,38.8,2010,12,31
364,723759,53990,2010-12-31,63.2,50.3,11.3,66.2,50.0,2010,12,31
364,162420,99999,2010-12-31,45.4,38.7,4.9,50.0,38.1,2010,12,31
364,478400,99999,2010-12-31,34.1,20.2,17.1,39.2,30.2,2010,12,31
364,723555,93940,2010-12-31,56.1,49.3,8.8,62.6,46.4,2010,12,31
...,...,...,...,...,...,...,...,...,...,...,...
360,717270,99999,2018-12-31,9.7,5.1,1.9,19.4,1.0,2018,12,31
209,355220,99999,2018-12-31,18.9,15.4,2.4,21.6,11.8,2018,12,31
208,355630,99999,2018-12-31,2.0,-3.7,999.9,11.8,-4.2,2018,12,31
358,131160,99999,2018-12-31,44.5,31.1,6.8,50.0,36.1,2018,12,31


In [42]:
station_loc

,USAF,WBAN,CTRY,LAT,LON,ELEV(M),LBL
11,010010,99999,NO,70.933,-8.667,9.0,"JAN MAYEN(NOR-NAVY), NO<br>Elevation: 9.0 m"
13,010014,99999,NO,59.792,5.341,48.8,"SORSTOKKEN, NO<br>Elevation: 48.8 m"
17,010020,99999,NO,80.050,16.250,8.0,"VERLEGENHUKEN, NO<br>Elevation: 8.0 m"
18,010030,99999,NO,77.000,15.500,12.0,"HORNSUND, NO<br>Elevation: 12.0 m"
21,010060,99999,NO,78.250,22.817,14.0,"EDGEOYA, NO<br>Elevation: 14.0 m"
...,...,...,...,...,...,...,...
29749,A00019,53969,US,34.226,-99.284,385.6,"WILBARGER COUNTY AIRPORT, TX, US<br>Elevation:..."
29750,A00021,63884,US,37.601,-81.559,543.5,"KEE FIELD AIRPORT, WV, US<br>Elevation: 543.5 m"
29752,A00023,63890,US,30.350,-81.883,30.2,"WHITEHOUSE NAVAL OUTLYING FIELD, FL, US<br>Ele..."
29757,A00031,3725,US,38.994,-79.144,292.6,"GRANT COUNT AIRPORT, WV, US<br>Elevation: 292.6 m"


In [43]:
merged_df = pd.merge(df_day, station_loc, how="outer", on="USAF")
merged_df

,USAF,WBAN_x,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY,WBAN_y,CTRY,LAT,LON,ELEV(M),LBL
0,161490,99999.0,2010-12-31,40.7,38.8,1.8,42.8,38.8,2010.0,12.0,31.0,99999,IT,44.020,12.612,12.5,"RIMINI, IT<br>Elevation: 12.5 m"
1,161490,99999.0,2011-12-31,38.2,28.5,2.1,50.0,29.7,2011.0,12.0,31.0,99999,IT,44.020,12.612,12.5,"RIMINI, IT<br>Elevation: 12.5 m"
2,161490,99999.0,2012-12-31,34.7,32.3,2.2,46.8,26.6,2012.0,12.0,31.0,99999,IT,44.020,12.612,12.5,"RIMINI, IT<br>Elevation: 12.5 m"
3,161490,99999.0,2013-12-31,46.2,41.5,3.9,50.0,42.8,2013.0,12.0,31.0,99999,IT,44.020,12.612,12.5,"RIMINI, IT<br>Elevation: 12.5 m"
4,161490,99999.0,2014-12-31,38.1,25.4,15.6,41.0,32.0,2014.0,12.0,31.0,99999,IT,44.020,12.612,12.5,"RIMINI, IT<br>Elevation: 12.5 m"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302206,A00019,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53969,US,34.226,-99.284,385.6,"WILBARGER COUNTY AIRPORT, TX, US<br>Elevation:..."
302207,A00021,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63884,US,37.601,-81.559,543.5,"KEE FIELD AIRPORT, WV, US<br>Elevation: 543.5 m"
302208,A00023,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63890,US,30.350,-81.883,30.2,"WHITEHOUSE NAVAL OUTLYING FIELD, FL, US<br>Ele..."
302209,A00031,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3725,US,38.994,-79.144,292.6,"GRANT COUNT AIRPORT, WV, US<br>Elevation: 292.6 m"


In [54]:
merged_clean_df = merged_df.loc[merged_df["USAF"] != "999999", :]
merged_clean_df.dropna(how='any')
us_merged_df = merged_clean_df.loc[merged_df["CTRY"] == "US", :]
us_merged_df

,USAF,WBAN_x,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY,WBAN_y,CTRY,LAT,LON,ELEV(M),LBL
9,723759,53990.0,2010-12-31,63.2,50.3,11.3,66.2,50.0,2010.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
10,723759,53990.0,2011-12-31,48.6,38.3,5.9,68.0,32.0,2011.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
11,723759,53990.0,2012-12-31,37.6,30.9,5.6,41.0,35.6,2012.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
12,723759,53990.0,2013-12-31,32.9,23.3,2.4,51.8,21.2,2013.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
13,723759,53990.0,2014-12-31,34.5,25.8,3.4,39.2,30.2,2014.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302206,A00019,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53969,US,34.226,-99.284,385.6,"WILBARGER COUNTY AIRPORT, TX, US<br>Elevation:..."
302207,A00021,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63884,US,37.601,-81.559,543.5,"KEE FIELD AIRPORT, WV, US<br>Elevation: 543.5 m"
302208,A00023,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63890,US,30.350,-81.883,30.2,"WHITEHOUSE NAVAL OUTLYING FIELD, FL, US<br>Ele..."
302209,A00031,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3725,US,38.994,-79.144,292.6,"GRANT COUNT AIRPORT, WV, US<br>Elevation: 292.6 m"


In [102]:
new_df=us_merged_df.dropna(how='any')

In [103]:
new_df

,USAF,WBAN_x,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY,WBAN_y,CTRY,LAT,LON,ELEV(M),LBL
9,723759,53990.0,2010-12-31,63.2,50.3,11.3,66.2,50.0,2010.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
10,723759,53990.0,2011-12-31,48.6,38.3,5.9,68.0,32.0,2011.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
11,723759,53990.0,2012-12-31,37.6,30.9,5.6,41.0,35.6,2012.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
12,723759,53990.0,2013-12-31,32.9,23.3,2.4,51.8,21.2,2013.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
13,723759,53990.0,2014-12-31,34.5,25.8,3.4,39.2,30.2,2014.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301564,724856,99999.0,2017-12-31,35.1,13.4,2.8,46.4,26.6,2017.0,12.0,31.0,99999,US,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
301565,724856,99999.0,2018-12-31,32.1,19.8,11.0,41.0,24.8,2018.0,12.0,31.0,99999,US,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
301599,725835,24119.0,2017-12-31,36.7,21.4,3.7,53.6,23.0,2017.0,12.0,31.0,24119,US,40.612,-116.892,1373.1,"BATTLE MOUNTAIN AIRPORT, NV, US<br>Elevation: ..."
301600,725835,24119.0,2018-12-31,27.9,21.5,7.5,33.8,19.4,2018.0,12.0,31.0,24119,US,40.612,-116.892,1373.1,"BATTLE MOUNTAIN AIRPORT, NV, US<br>Elevation: ..."


In [112]:
# us_drop_na_df = us_merged_df.dropna(how='any')
# us_drop_na_df

In [107]:
new_df

,USAF,WBAN_x,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY,WBAN_y,CTRY,LAT,LON,ELEV(M),LBL
9,723759,53990.0,2010-12-31,63.2,50.3,11.3,66.2,50.0,2010.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
10,723759,53990.0,2011-12-31,48.6,38.3,5.9,68.0,32.0,2011.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
11,723759,53990.0,2012-12-31,37.6,30.9,5.6,41.0,35.6,2012.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
12,723759,53990.0,2013-12-31,32.9,23.3,2.4,51.8,21.2,2013.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
13,723759,53990.0,2014-12-31,34.5,25.8,3.4,39.2,30.2,2014.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301564,724856,99999.0,2017-12-31,35.1,13.4,2.8,46.4,26.6,2017.0,12.0,31.0,99999,US,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
301565,724856,99999.0,2018-12-31,32.1,19.8,11.0,41.0,24.8,2018.0,12.0,31.0,99999,US,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
301599,725835,24119.0,2017-12-31,36.7,21.4,3.7,53.6,23.0,2017.0,12.0,31.0,24119,US,40.612,-116.892,1373.1,"BATTLE MOUNTAIN AIRPORT, NV, US<br>Elevation: ..."
301600,725835,24119.0,2018-12-31,27.9,21.5,7.5,33.8,19.4,2018.0,12.0,31.0,24119,US,40.612,-116.892,1373.1,"BATTLE MOUNTAIN AIRPORT, NV, US<br>Elevation: ..."


In [1]:
for i range(len(new_df)):
    print(i)
#     row = new_df[i]

SyntaxError: invalid syntax (<ipython-input-1-17154b909896>, line 1)

In [113]:
# us_drop_na_df['YEARMODA']

In [114]:
# list(us_drop_na_df['YEARMODA'])

In [108]:
list(new_df['YEARMODA'])

[Timestamp('2010-12-31 00:00:00'),
 Timestamp('2011-12-31 00:00:00'),
 Timestamp('2012-12-31 00:00:00'),
 Timestamp('2013-12-31 00:00:00'),
 Timestamp('2014-12-31 00:00:00'),
 Timestamp('2015-12-31 00:00:00'),
 Timestamp('2016-12-31 00:00:00'),
 Timestamp('2017-12-31 00:00:00'),
 Timestamp('2018-12-31 00:00:00'),
 Timestamp('2010-12-31 00:00:00'),
 Timestamp('2011-12-31 00:00:00'),
 Timestamp('2012-12-31 00:00:00'),
 Timestamp('2013-12-31 00:00:00'),
 Timestamp('2014-12-31 00:00:00'),
 Timestamp('2015-12-31 00:00:00'),
 Timestamp('2016-12-31 00:00:00'),
 Timestamp('2017-12-31 00:00:00'),
 Timestamp('2018-12-31 00:00:00'),
 Timestamp('2010-12-30 00:00:00'),
 Timestamp('2011-12-30 00:00:00'),
 Timestamp('2012-12-28 00:00:00'),
 Timestamp('2013-12-31 00:00:00'),
 Timestamp('2014-12-31 00:00:00'),
 Timestamp('2015-12-23 00:00:00'),
 Timestamp('2016-12-30 00:00:00'),
 Timestamp('2017-12-21 00:00:00'),
 Timestamp('2018-12-31 00:00:00'),
 Timestamp('2010-12-31 00:00:00'),
 Timestamp('2011-12-

In [110]:

new_df.dtypes

USAF                object
WBAN_x             float64
YEARMODA    datetime64[ns]
TEMP               float64
DEWP               float64
WDSP               float64
MAX                float64
MIN                float64
YEAR               float64
MONTH              float64
DAY                float64
WBAN_y               int64
CTRY                object
LAT                float64
LON                float64
ELEV(M)            float64
LBL                 object
dtype: object

In [111]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.USWeather
collection = db.weather
db.collection.insert_many(new_df.to_dict('records'))

'2018-04-14'